# open problems (task batch correction / label proj)


In [ ]:
import pandas as pd
import requests
import json
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection
import scanpy as sc
from scprint import scPrint
from scdataloader import Preprocessor
from scprint.tasks import Embedder, FinetuneBatchClass
from scprint.tasks.cell_emb import compute_classification
import numpy as np

%load_ext autoreload
%autoreload 2

In [8]:
! uv pip list | grep scib #same version as OP

scib                       1.1.7
scib-metrics               0.5.6


In [ ]:
LOC = "/pasteur/appa/scratch/jkalfon/data/spcrint_data/"

In [3]:
url = "https://raw.githubusercontent.com/openproblems-bio/website/main/results/batch_integration/data/results.json"
response = requests.get(url)

with open("results_batch.json", "w") as f:
    f.write(response.text)

url = "https://raw.githubusercontent.com/openproblems-bio/website/main/results/label_projection/data/results.json"
response = requests.get(url)

with open("results_label.json", "w") as f:
    f.write(response.text)

print("File downloaded successfully!")

File downloaded successfully!


In [4]:
print("File downloaded successfully!")
with open("results_batch.json", "r") as f:
    data_batch = json.load(f)

File downloaded successfully!


In [5]:
res = {}
for dataset in data_batch:
    dataset_id = dataset["dataset_id"]
    if dataset_id not in res:
        res[dataset_id] = {}
    res[dataset_id].update({dataset["method_id"]: dataset["metric_values"]})

In [6]:
with open("results_label.json", "r") as f:
    data_label = json.load(f)

In [7]:
res_label = {}
for dataset in data_label:
    dataset_id = dataset["dataset_id"]
    if dataset_id not in res_label:
        res_label[dataset_id] = {}
    res_label[dataset_id].update({dataset["method_id"]: dataset["metric_values"]})

In [15]:
res_label.keys()

dict_keys(['cellxgene_census/dkd', 'cellxgene_census/gtex_v9', 'cellxgene_census/hypomap', 'cellxgene_census/immune_cell_atlas', 'cellxgene_census/mouse_pancreas_atlas', 'cellxgene_census/tabula_sapiens', None])

In [8]:
pd.DataFrame(res_label["cellxgene_census/dkd"])

,knn,logistic_regression,majority_vote,mlp,naive_bayes,random_labels,scanvi,scanvi_scarches,scgpt_zeroshot,scimilarity,scimilarity_knn,seurat_transferdata,singler,true_labels,uce,xgboost,geneformer,scgpt_finetuned,scprint
accuracy,0.9490,0.9572,0.2954,0.9540,0.9269,0.1808,0.9570,0.9570,0.8486,0.8869,0.9553,0.9541,0.9147,1,0.1813,0.9644,NA,NA,NA
f1_macro,0.9286,0.9413,0.0351,0.9245,0.9181,0.0774,0.9360,0.9366,0.5239,0.6233,0.9292,0.9344,0.9027,1,0.0743,0.9225,NA,NA,NA
f1_micro,0.9490,0.9572,0.2954,0.9540,0.9269,0.1808,0.9570,0.9570,0.8486,0.8869,0.9553,0.9541,0.9147,1,0.1813,0.9644,NA,NA,NA
f1_weighted,0.9487,0.9567,0.1347,0.9529,0.9296,0.1801,0.9579,0.9567,0.8339,0.8655,0.9556,0.9544,0.9192,1,0.1790,0.9634,NA,NA,NA


In [ ]:
pd.DataFrame(res["cellxgene_census/dkd"])

,batchelor_fastmnn,batchelor_mnn_correct,bbknn,combat,embed_cell_types,embed_cell_types_jittered,geneformer,harmony,harmonypy,liger,mnnpy,no_integration,no_integration_batch,pyliger,scalex,scanorama,scanvi,scgpt_zeroshot,scimilarity,scvi,shuffle_integration,shuffle_integration_by_batch,shuffle_integration_by_cell_type,uce,scgpt_finetuned,scprint
ari,0.6342,NA,0.6331,0.5976,1,1,0.0561,0.6101,0.6272,0.4991,NA,0.6958,0.288,0.54,0.5312,0.0517,0.8327,0.6689,0.677,0.7046,-0.0001,4.150000e-02,0.6970,0.6864,NA,NA
asw_batch,0.8696,NA,NA,0.8584,0.984,0.9827,0.4056,0.8814,0.8795,0.7698,NA,0.8521,0.6947,0.7818,0.8226,0.9097,0.873,0.906,0.8628,0.8991,0.9496,8.948000e-01,0.9323,0.9509,NA,NA
asw_label,0.5052,NA,NA,0.5014,0.979,0.979,0.2598,0.4946,0.5,0.4379,NA,0.5134,0.4363,0.4636,0.4674,0.4457,0.6243,0.5136,0.5531,0.5367,0.4827,4.489000e-01,0.5134,0.5241,NA,NA
cell_cycle_conservation,0.8279,NA,NA,0.8044,0.6453,0.6441,0.3616,0.7507,0.7525,0.5948,NA,0.8202,0.8171,0.6675,0.5774,0.1637,0.7464,0.7409,0.6673,0.6278,0.1379,1.540000e-01,0.5617,0.6976,NA,NA
clisi,0.9981,NA,0.9412,0.9993,1,1,0.9645,0.9959,0.9975,0.9867,NA,0.9996,0.999,0.9836,0.9903,0.9249,1,0.9975,0.9988,0.9995,0.8620,9.106000e-01,0.9996,0.999,NA,NA
graph_connectivity,0.8969,NA,0.9546,0.9436,1,1,0.081,0.8829,0.8815,0.6349,NA,0.9502,0.7334,0.639,0.7367,0.2440,0.985,0.9051,0.9155,0.966,0.0526,1.922000e-01,0.9512,0.9507,NA,NA
hvg_overlap,NA,NA,NA,0.5255,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.2410,0.2593,NA,NA,NA,NA,0.4995,1.000000e+00,0.6372,NA,NA,NA
ilisi,0.0737,NA,0.2703,0.0129,0.116,0.1162,0,0.1194,0.0986,0.193,NA,0.0317,0.0005,0.2059,0.0704,0.0905,0.0586,0.0819,0.078,0.0497,0.3065,3.200000e-02,0.1237,0.0648,NA,NA
isolated_label_asw,0.536,NA,NA,0.6147,0.979,0.979,0.4544,0.5958,0.5876,0.6283,NA,0.599,0.6479,0.6127,0.5733,0.4685,0.7019,0.5543,0.5948,0.6288,0.4836,4.477000e-01,0.5990,0.5215,NA,NA
isolated_label_f1,0.018,NA,0.0142,0.1021,1,1,0.0802,0.0256,0.0737,0.0741,NA,0.0896,0.0888,0.0448,0.0137,0.0114,0.0908,0.0213,0.0164,0.1041,0.0030,7.500000e-03,0.0895,0.0242,NA,NA


In [11]:
model_checkpoint_file = "../models/ogvvg2z7-v1.ckpt"

In [13]:
model = scPrint.load_from_checkpoint(
    model_checkpoint_file, precpt_gene_emb=None, gene_pos_file=None
)
model = model.to("cuda")

FYI: scPrint is not attached to a `Trainer`.


In [10]:
datasets = {
    "cellxgene_census/dkd": "https://datasets.cellxgene.cziscience.com/46d8d92b-32e0-4ca5-9907-4dbf519c7fc3.h5ad",  # 0.3
    "cellxgene_census/gtex_v9": "https://datasets.cellxgene.cziscience.com/002308e1-0121-4aa1-b8f2-9d034cf44b0f.h5ad",  # 1gb
    "cellxgene_census/hypomap": "https://datasets.cellxgene.cziscience.com/d3be7423-d664-4913-89a9-a506cae4c28f.h5ad",  # 4gb
    "cellxgene_census/mouse_pancreas_atlas": "https://datasets.cellxgene.cziscience.com/49243c50-bf0c-4b10-87f8-55ec9f455399.h5ad",  # 4gb
    "cellxgene_census/immune_cell_atlas": "https://datasets.cellxgene.cziscience.com/78819b62-0699-4672-8dc8-d9317b04d255.h5ad",  # 3gb
    # 'cellxgene_census/tabula_sapiens': 'https://datasets.cellxgene.cziscience.com/5a495302-b7cd-4bf9-853e-95627b00bb03.h5ad' # 42gb
}

LOC = "/pasteur/appa/scratch/jkalfon/data/spcrint_data/temp/"  # "../../data/temp/"  #

In [17]:
metrics = {}
for name, url in list(datasets.items())[:]:
    print("doing ", name)
    adata = sc.read(LOC + name + ".h5ad", backup_url=url)
    preprocessor = Preprocessor(
        force_preprocess=True,
        skip_validate=True,
        # drop_non_primary=False,
        is_symbol=False,
        do_postp=model.expr_emb_style == "metacell",
    )
    print("")
    adata = preprocessor(adata)
    if model.expr_emb_style == "metacell":
        sc.pp.neighbors(adata, use_rep="X_pca")

    embed = Embedder(
        how="random expr",
        max_len=3000,
        num_workers=8,
        pred_embedding=["cell_type_ontology_term_id"],
        doplot=False,
    )
    n_adata, _ = embed(model, adata)
    # cls regular
    loc = n_adata.obs.columns[n_adata.obs.columns.str.startswith("CL:")]
    pred = n_adata.obs.loc[:, loc]
    n_adata.obs["pred_cell_type_ontology_term_id"] = loc[pred.values.argmax(1)].values
    n_adata.obs["_ref_cls"] = loc[pred.values.argmax(1)].values
    metrics[name + "_ref_cls"] = compute_classification(
        n_adata,
        ["cell_type_ontology_term_id"],
        label_decoders=model.label_decoders,
        labels_hierarchy=model.labels_hierarchy,
    )
    # cls ref
    for i in range(3):
        pred.iloc[:, :] = zero_shot_annotation_with_refinement(
            pred.values, n_adata, return_raw=True
        ).astype(np.float32)
    n_adata.obs["pred_cell_type_ontology_term_id"] = loc[
        zero_shot_annotation_with_refinement(pred.values, n_adata)
    ].values
    metrics[name + "_smooth_cls"] = compute_classification(
        n_adata,
        ["cell_type_ontology_term_id"],
        label_decoders=model.label_decoders,
        labels_hierarchy=model.labels_hierarchy,
    )
    # cls cluster
    # sc.tl.leiden(n_adata, resolution=4.0)
    for i in n_adata.obs["leiden"].unique():
        n_adata.obs.loc[
            n_adata.obs["leiden"] == str(i), "pred_cell_type_ontology_term_id"
        ] = loc[pred[n_adata.obs["leiden"] == str(i)].values.sum(0).argsort()[::-1][0]]
    metrics[name + "_clust_cls"] = compute_classification(
        n_adata,
        ["cell_type_ontology_term_id"],
        label_decoders=model.label_decoders,
        labels_hierarchy=model.labels_hierarchy,
    )

    bm = Benchmarker(
        adata,
        batch_key="donor_id",  # "batch",  # batch, tech, assay_ontology_term_id, donor_id
        label_key="cell_type",  # celltype
        embedding_obsm_keys=["scprint_emb"],
        bio_conservation_metrics=BioConservation(),
        batch_correction_metrics=BatchCorrection(),
        n_jobs=10,
    )
    bm.benchmark()
    print(bm.get_results())
    metrics[name + "_batch_corr"] = bm.get_results()
    bm.plot_results_table(min_max_scale=False)

doing  cellxgene_census/dkd

Dropping layers:  KeysView(Layers with keys: )
checking raw counts
removed 0 non primary cells, 39176 renamining
filtered out 0 cells, 39176 renamining
Removed 2 genes not known to the ontology
Removed 0 duplicate genes
Added 34649 genes in the ontology but not present in the dataset
starting QC
Seeing 3477 outliers (8.88% of total dataset):
done
AnnData object with n_obs × n_vars = 39176 × 70116
    obs: 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'sample_uuid', 'sample_preservation_method', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'suspension_uuid', 'suspension_type', 'library_uuid', 'assay_ontology_term_id', 'mapped_reference_annotation', 'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type', 'disease_ontology_term_id', 'reported_diseases', 'sex_ontology_term_id', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rpl', 'percent.rps', 'doublet_id', 'nCount_SCT', 'nFeature_SCT', 'seurat_clusters', 'tissu

/pasteur/appa/homes/jkalfon/scdataloader/scdataloader/utils.py:427: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  organismdf = pd.concat(organismdf)


not on wandb, could not set name


  0%|          | 0/613 [00:00<?, ?it/s]

> /pasteur/appa/homes/jkalfon/scPRINT/scprint/model/model.py(2204)_predict()
   2202             # seq len must be a multiple of 128
   2203             num = (1 if self.use_metacell_token else 0) + (
-> 2204                 (len(self.classes) + 1) if not self.cell_transformer else 0
   2205             )
   2206             if (expression.shape[1] + num) % 128 != 0:



  0%|          | 0/613 [00:31<?, ?it/s]


# same with scPRINT-V1


# same with finetuning batch


In [ ]:
## ISSUE: many batches to correct, mmd might not be the right tool

## same with fine tuning class


In [ ]:
finetuner = FinetuneBatchClass(
    batch_key="donor_id",
    max_len=3000,
)

model, metrics[name + "_fine_tuning"] = finetuner(model=model, train_adata=adata)

In [ ]:
metrics